In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import numpy as np
from random import randint, uniform
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor, NearestNeighbors
from sklearn.metrics.cluster import normalized_mutual_info_score
from datetime import datetime
from sklearn.metrics.cluster import normalized_mutual_info_score


In [0]:
class Kmeans:


    def __init__(self, k, points):
      self.k=k

      self.points = points
      self.centroids=np.full(shape=(self.k,self.points.shape[1]), fill_value=-1, dtype=np.float64)

      # auxiliary columns: point_id, assigned cluster, distance to cluster
      points_aux=np.full(shape=(self.points.shape[0], 3),fill_value=-1)
      points_aux[:, 0]=np.arange(start=0, stop=self.points.shape[0])
      self.points=np.append(self.points, points_aux, axis=1)
      
      # auxiliary columns: centroid id, counts of assigned points
      centroids_aux=np.full(shape=(self.k, 2),fill_value=-1)
      centroids_aux[:, 0]=np.arange(start=0, stop=self.k)
      self.centroids=np.append(self.centroids,centroids_aux,axis=1 )


    # method used in Lloyd and initialization 
    def calculate_all_centroids(self):
      for k in range(self.k):
        self.centroids[k,:-2] = self.points[self.points[:,-2]==k][:, :-3].mean(axis=0)
        self.centroids[k,-1] = len(self.points[self.points[:,-2]==k][:, -3])
    

    def initialize_clusters(self, method='forgy'):
      self.points[:,-2]=-1
      if method=='forgy':
        while any(k not in self.points[:,-2] for k in range(0,self.k)):
          self.points[:,-2]=np.random.randint(0, self.k, self.points.shape[0])
      self.calculate_all_centroids()


    # method used in MacQueen - recalculates old and new centroid
    def calculate_two_centroids(self, point, min_cen):
      old = np.where(self.centroids[:,-2]==point[-2])[0]
                            
      #update old centroid
      self.centroids[old,:-2] = self.centroids[old,:-2] * self.centroids[old,-1] - point[:-3]                
      self.centroids[old,-1] = self.centroids[old,-1] - 1
      self.centroids[old,:-2] = self.centroids[old,:-2] / self.centroids[old,-1]

      #update new centroid
      min_cen[:-2] = min_cen[:-2] * min_cen[-1] + point[:-3]
      min_cen[-1] = min_cen[-1] + 1
      min_cen[:-2] = min_cen[:-2] / min_cen[-1]
      return min_cen


    def assign_clusters(self, macqueen=False):
      for point in self.points:
        min=np.inf
        min_cen=point[-2]
        for cen in self.centroids:
            diff = point[:-3] - cen[:-2]
            dist = np.inner(diff,diff)
            if dist < min:
                min = dist
                min_cen = cen

        if min_cen[-2]!=point[-2]:
          if macqueen==True:
            min_cen = self.calculate_two_centroids(point, min_cen)
          #update centroid assignment if changed
          point[-2]=min_cen[-2]
          point[-1]=min 



    def plot_clusters(self, points, centroids, i):
        #select subset of dataframe with features
        X = points[:,:-3]
        y = points[:,-2]
       
        X_c = centroids[:,:-2]
        y_c = centroids[:,-2]
       
        if X.shape[1] == 2:
            method = 'Plot'
            X_reduced_pca = X
            X_c_reduced_pca = X_c
        else:
            method = 'PCA'
            # calculate PCA Implementation of dim reducition
            pca=PCA(n_components=2, random_state=42)
            pca.fit(X)
            X_reduced_pca = pca.transform(X)
            X_c_reduced_pca = pca.transform(X_c)

        #plotting
        f, (ax1) = plt.subplots(1, 1, figsize=(6,6))
        f.suptitle('Iteration ' + str(i), fontsize=14)

        # scatter plot
        ax1.scatter(X_reduced_pca[:,0], X_reduced_pca[:,1],
                    c=y, cmap='rainbow', linewidths=4)
        ax1.scatter(X_c_reduced_pca[:,0], X_c_reduced_pca[:,1],
                    c=y_c, marker='x',
                    s=150, cmap='rainbow', linewidths=10)
        ax1.set_title(method, fontsize=14)
        ax1.grid(True)

        plt.show()


    def MacQueen(self, make_plots=False):
        i=0
        while True:
          print('ITERATION', i, datetime.now())
          if make_plots:
            self.plot_clusters(self.points, self.centroids, i)   

          clusters_before = self.points[:,-2].copy()
          self.assign_clusters(macqueen=True)
          clusters_after = self.points[:,-2].copy()
          #check if algorithm converged
          if not any(c_before != c_after for c_before, c_after in zip(clusters_before,clusters_after)):
            break
          #self.calculate_all_centroids()
          i=i+1


    def Lloyd(self, make_plots=False):
      i=0
      while True:
          print('ITERATION', i, datetime.now())
          if make_plots:
            self.plot_clusters(self.points, self.centroids, i)         
          clusters_before = self.points[:,-2].copy()
          self.assign_clusters(macqueen=False)
          clusters_after = self.points[:,-2].copy()
          #check if algorithm converged
          if not any(c_before != c_after for c_before, c_after in zip(clusters_before,clusters_after)):
              break
          self.calculate_all_centroids()
          i=i+1
                      

In [4]:
#load the HTRU_2 dataset and save true class labels as separate array

file_list = drive.ListFile(
    {'q': "'1y2Ypp16D8hE959IrKCADOvpSHz4Tu2Mz' in parents and trashed=false"}).GetList()
for file in file_list:
  downloaded = drive.CreateFile({'id':file['id']})   
  downloaded.GetContentFile(file['originalFilename'])
  print(file['originalFilename'], file['id'])

data_htru_raw = np.genfromtxt('HTRU_2.csv', delimiter=',')
data_htru=data_htru_raw[:, 0:-1].copy()  

data_skin_raw=np.genfromtxt('Skin_NonSkin.txt', delimiter='\t')
data_skin=data_skin_raw[:, 0:-1].copy()  



Skin_NonSkin.txt 1ZbOE3GxszqNT6iMqxkD_Fdh4y2yzm-hY
HTRU_2.csv 1RPjRbzuf4SZ5VunY1HgQVlWfZ7dpjOTA


In [8]:
# Own implementation testing:
ex4=Kmeans(2, data_htru)
ex4.initialize_clusters()
ex4.MacQueen(make_plots=False)
own_NMI= normalized_mutual_info_score(data_htru_raw[:,-1], ex4.points[:,-2])
print(own_NMI)

#Sklearn clustering to compare
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit_predict(data_htru)
sklearn_NMI=normalized_mutual_info_score(data_htru_raw[:,8], kmeans)

print(sklearn_NMI)

ITERATION 0 2020-03-22 12:49:52.153202
ITERATION 1 2020-03-22 12:49:52.617683
ITERATION 2 2020-03-22 12:49:52.802889
ITERATION 3 2020-03-22 12:49:52.968688
ITERATION 4 2020-03-22 12:49:53.114894
ITERATION 5 2020-03-22 12:49:53.260394
ITERATION 6 2020-03-22 12:49:53.394365
ITERATION 7 2020-03-22 12:49:53.544444
ITERATION 8 2020-03-22 12:49:53.674894
ITERATION 9 2020-03-22 12:49:53.808721
ITERATION 10 2020-03-22 12:49:53.937517
ITERATION 11 2020-03-22 12:49:54.071056
ITERATION 12 2020-03-22 12:49:54.207138
ITERATION 13 2020-03-22 12:49:54.340502
ITERATION 14 2020-03-22 12:49:54.473557
ITERATION 15 2020-03-22 12:49:54.608775
ITERATION 16 2020-03-22 12:49:54.737772
0.02649673183252321
0.02643778644792407


In [17]:
data_skin_raw[:,-1]-1

array([0., 0., 0., ..., 1., 1., 1.])